In [15]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/esoh/"
oCV_DIR = r"F:/Git Hub rep/CRC/neew/PyBaMM/Untitled Folder/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

from scipy.io import savemat


In [16]:
## Added from old vrsion 


In [17]:
parameter_values = get_parameter_values()

In [18]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
    }
)
# spm.print_parameter_info()
param=spm.param

In [19]:
cell = 1

In [20]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)
print(Temp)

25


In [21]:
# pybamm.set_logging_level("WARNING")
pybamm.set_logging_level("NOTICE")

drive_cycle = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_default_5Ah.csv", comment="#", header=None).to_numpy()
drive_cycle2 = pd.read_csv(r"C:\Users\Hamidreza\Desktop\CRC\Drive Cycle\US06_GM_5Ah.csv", comment="#", header=None).to_numpy()

experiment = pybamm.Experiment( # defult D.C. DRC-DRC *1.25
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1.9 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment2 = pybamm.Experiment( #lab cycling
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *300,
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

## GM_DCRDCR
experiment3 = pybamm.Experiment( #GM_ DRC-DRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for .6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Discharge at C/4 for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment4 = pybamm.Experiment( #Gm-RCRC
    [
        ("Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.6 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06_GM_5Ah (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.1 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06_GM_5Ah": drive_cycle2},
    termination="50% capacity",
#     cccv_handling="ode",
)

experiment5 = pybamm.Experiment( # defult D.C. RC-RC
    [
        ("Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/4 until 4.2V",
         "Hold at 4.2V for 1.8 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes", 
         # Home
         "Run US06 (A) for 1 hours",
         "Rest for 5 minutes",
         "Rest for 1 hours", # should be P/4 
         "Rest for 5 minutes",
         "Charge at C/8 until 4.2V",
          "Hold at 4.2V for 3.7 hours",  # needs fixing : last step +this step =2.5 hrs
         "Rest for 20 minutes",     
        )

    ]*540,
    drive_cycles={"US06": drive_cycle},
    termination="50% capacity",
#     cccv_handling="ode",
)

In [22]:
parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Positive electrode LAM constant proportional term [s-1]": 2.5112e-07,
        "Negative electrode LAM constant proportional term [s-1]": 2.4741e-06,
        "Positive electrode LAM constant exponential term": 1.1942,
        "Negative electrode LAM constant exponential term": 1.1942,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
        "SEI kinetic rate constant [m.s-1]": 1.08494281e-16,
        "EC diffusivity [m2.s-1]": 8.30909086e-19,
        "SEI growth activation energy [J.mol-1]": 1.58777981e+04,
        # "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        # "EC diffusivity [m2.s-1]": 2e-18,
        # "SEI growth activation energy [J.mol-1]": 0,
    },
    check_already_exists=False,
)


In [23]:
# all_sumvars_dict1 = cycle_adaptive_simulation(spm, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [24]:
sim_long = pybamm.Simulation(spm, experiment=experiment3, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe"))
sol_long = sim_long.solve(initial_soc=SOC_0, save_at_cycles=10  )

2022-10-23 20:18:30.500 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/540 (18.415 ms elapsed) --------------------
2022-10-23 20:18:30.502 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 1/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:18:31.759 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 2/14: Rest for 5 minutes
2022-10-23 20:18:31.848 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 3/14: Discharge at C/4 for 1 hours
2022-10-23 20:18:32.640 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 4/14: Rest for 5 minutes
2022-10-23 20:18:32.660 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:18:33.103 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 6/14: Hold at 4.2V for .6 hours
2022-10-23 20:18:33.289 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 7/14: Rest for 20 minutes
2022-10-23 20:18:33.310 - [NOTICE] callbacks.on_step_start(182): Cycle 1/540, step 8/14:

2022-10-23 20:18:49.463 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 8/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:18:50.122 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 9/14: Rest for 5 minutes
2022-10-23 20:18:50.134 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 10/14: Discharge at C/4 for 1 hours
2022-10-23 20:18:50.196 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 11/14: Rest for 5 minutes
2022-10-23 20:18:50.208 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 20:18:50.430 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 13/14: Hold at 4.2V for 1 hours
2022-10-23 20:18:50.552 - [NOTICE] callbacks.on_step_start(182): Cycle 5/540, step 14/14: Rest for 20 minutes
2022-10-23 20:18:52.308 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.830 Ah (originally 4.858 Ah, will stop at 2.429 Ah)
2022-10-23 20:18:52.308 - [NOTICE] callbacks.on_cycle_start(174): Cyc

2022-10-23 20:19:06.729 - [NOTICE] callbacks.on_cycle_start(174): Cycle 10/540 (36.251 s elapsed) --------------------
2022-10-23 20:19:06.729 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 1/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:19:07.340 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 2/14: Rest for 5 minutes
2022-10-23 20:19:07.351 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 3/14: Discharge at C/4 for 1 hours
2022-10-23 20:19:07.402 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 4/14: Rest for 5 minutes
2022-10-23 20:19:07.416 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:19:07.520 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 6/14: Hold at 4.2V for .6 hours
2022-10-23 20:19:07.602 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, step 7/14: Rest for 20 minutes
2022-10-23 20:19:07.622 - [NOTICE] callbacks.on_step_start(182): Cycle 10/540, st

2022-10-23 20:19:22.601 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 7/14: Rest for 20 minutes
2022-10-23 20:19:22.628 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 8/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:19:23.230 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 9/14: Rest for 5 minutes
2022-10-23 20:19:23.240 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 10/14: Discharge at C/4 for 1 hours
2022-10-23 20:19:23.296 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 11/14: Rest for 5 minutes
2022-10-23 20:19:23.308 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 20:19:23.836 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 13/14: Hold at 4.2V for 1 hours
2022-10-23 20:19:23.957 - [NOTICE] callbacks.on_step_start(182): Cycle 14/540, step 14/14: Rest for 20 minutes
2022-10-23 20:19:25.682 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.764 

2022-10-23 20:19:38.406 - [NOTICE] callbacks.on_step_start(182): Cycle 18/540, step 14/14: Rest for 20 minutes
2022-10-23 20:19:40.117 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 4.734 Ah (originally 4.858 Ah, will stop at 2.429 Ah)
2022-10-23 20:19:40.117 - [NOTICE] callbacks.on_cycle_start(174): Cycle 19/540 (1 minute, 10 seconds elapsed) --------------------
2022-10-23 20:19:40.117 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 1/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:19:40.737 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 2/14: Rest for 5 minutes
2022-10-23 20:19:40.757 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 3/14: Discharge at C/4 for 1 hours
2022-10-23 20:19:40.806 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 4/14: Rest for 5 minutes
2022-10-23 20:19:40.816 - [NOTICE] callbacks.on_step_start(182): Cycle 19/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:19:40.922 - [NOTICE] callbacks.on_st

...

2022-10-23 20:32:12.354 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 3/14: Discharge at C/4 for 1 hours
2022-10-23 20:32:12.405 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 4/14: Rest for 5 minutes
2022-10-23 20:32:12.423 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:32:12.521 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 6/14: Hold at 4.2V for .6 hours
2022-10-23 20:32:12.601 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 7/14: Rest for 20 minutes
2022-10-23 20:32:12.621 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 8/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:32:13.260 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 9/14: Rest for 5 minutes
2022-10-23 20:32:13.270 - [NOTICE] callbacks.on_step_start(182): Cycle 225/540, step 10/14: Discharge at C/4 for 1 hours
2022-10-23 20:32:13.320 - [NOTICE] callbacks.on_step_start(182): Cycle 2

2022-10-23 20:32:28.039 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 9/14: Rest for 5 minutes
2022-10-23 20:32:28.049 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 10/14: Discharge at C/4 for 1 hours
2022-10-23 20:32:28.099 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 11/14: Rest for 5 minutes
2022-10-23 20:32:28.118 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 20:32:28.314 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 13/14: Hold at 4.2V for 1 hours
2022-10-23 20:32:28.434 - [NOTICE] callbacks.on_step_start(182): Cycle 229/540, step 14/14: Rest for 20 minutes
2022-10-23 20:32:30.258 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.888 Ah (originally 4.858 Ah, will stop at 2.429 Ah)
2022-10-23 20:32:30.258 - [NOTICE] callbacks.on_cycle_start(174): Cycle 230/540 (14 minutes, 0 seconds elapsed) --------------------
2022-10-23 20:32:30.258 - [NOTICE] callbacks.

2022-10-23 20:32:44.891 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.840 Ah (originally 4.858 Ah, will stop at 2.429 Ah)
2022-10-23 20:32:44.891 - [NOTICE] callbacks.on_cycle_start(174): Cycle 234/540 (14 minutes, 14 seconds elapsed) --------------------
2022-10-23 20:32:44.891 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 1/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:32:45.545 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 2/14: Rest for 5 minutes
2022-10-23 20:32:45.555 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 3/14: Discharge at C/4 for 1 hours
2022-10-23 20:32:45.601 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 4/14: Rest for 5 minutes
2022-10-23 20:32:45.611 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:32:45.726 - [NOTICE] callbacks.on_step_start(182): Cycle 234/540, step 6/14: Hold at 4.2V for .6 hours
2022-10-23 20:32:45.800 - [NOTICE] c

2022-10-23 20:33:00.301 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 5/14: Charge at C/4 until 4.2V
2022-10-23 20:33:00.397 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 6/14: Hold at 4.2V for .6 hours
2022-10-23 20:33:00.474 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 7/14: Rest for 20 minutes
2022-10-23 20:33:00.494 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 8/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:33:01.156 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 9/14: Rest for 5 minutes
2022-10-23 20:33:01.166 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 10/14: Discharge at C/4 for 1 hours
2022-10-23 20:33:01.216 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 11/14: Rest for 5 minutes
2022-10-23 20:33:01.226 - [NOTICE] callbacks.on_step_start(182): Cycle 238/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 20:33:01.415 - [NOTICE] callbacks.on_step_start(182): Cycle 238

2022-10-23 20:33:15.933 - [NOTICE] callbacks.on_step_start(182): Cycle 242/540, step 11/14: Rest for 5 minutes
2022-10-23 20:33:15.943 - [NOTICE] callbacks.on_step_start(182): Cycle 242/540, step 12/14: Charge at C/8 until 4.2V
2022-10-23 20:33:16.123 - [NOTICE] callbacks.on_step_start(182): Cycle 242/540, step 13/14: Hold at 4.2V for 1 hours
2022-10-23 20:33:16.251 - [NOTICE] callbacks.on_step_start(182): Cycle 242/540, step 14/14: Rest for 20 minutes
2022-10-23 20:33:18.073 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 2.721 Ah (originally 4.858 Ah, will stop at 2.429 Ah)
2022-10-23 20:33:18.073 - [NOTICE] callbacks.on_cycle_start(174): Cycle 243/540 (14 minutes, 48 seconds elapsed) --------------------
2022-10-23 20:33:18.073 - [NOTICE] callbacks.on_step_start(182): Cycle 243/540, step 1/14: Run US06_GM_5Ah (A) for 1 hours
2022-10-23 20:33:18.728 - [NOTICE] callbacks.on_step_start(182): Cycle 243/540, step 2/14: Rest for 5 minutes
2022-10-23 20:33:18.738 - [NOTICE] callbac

In [25]:
qwrqwf

NameError: name 'qwrqwf' is not defined

In [ ]:
pybamm.plot_summary_variables(sol_long)

In [ ]:
sim_long.plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#         "Electrolyte concentration [mol.m-3]",
#         "Positive particle surface concentration [mol.m-3]",
        "Current [A]",
#         "Negative electrode potential [V]",
#         "Electrolyte potential [V]",
#         "Positive electrode potential [V]",
        "Terminal voltage [V]",
#         "X-averaged cell temperature",
        "Negative electrode SOC",
        "Positive electrode SOC",
#         "SEI thickness [m]",
        "Loss of capacity to SEI [A.h]",
        "Negative electrode capacity [A.h]",
        "Positive electrode capacity [A.h]",
        "Discharge capacity [A.h]"

        
    ]
)

In [ ]:
print(list(sol_long.summary_variables.keys()))

In [ ]:
type(sol_long.summary_variables[list(sol_long.summary_variables.keys())[1]])

In [ ]:
# mdic={}
# for i in range (1):
# # range(len(sol_long.summary_variables)):
# #     stringing=''.join([list(sol_long.summary_variables.keys())[i]])
#     mdic={f"{i}": sol_long.summary_variables[list(sol_long.summary_variables.keys())[i]]}
# savemat("summary",mdic)


In [28]:
ident=""

In [30]:
x100=sol_long.summary_variables["x_100"]
y100=sol_long.summary_variables["y_100"]
x0=sol_long.summary_variables["x_0"]
y0=sol_long.summary_variables["y_0"]
Cap=sol_long.summary_variables["Capacity [A.h]"]
LoC_SEI_summ=sol_long.summary_variables["Loss of capacity to SEI [A.h]"]
LAM_P=sol_long.summary_variables["Loss of active material in positive electrode [%]"]
LAM_N=sol_long.summary_variables["Loss of active material in negative electrode [%]"]
C_Neg=sol_long.summary_variables["C_n"]
C_pos=sol_long.summary_variables["C_p"]
LLI=sol_long.summary_variables["Loss of lithium inventory [%]"]
cycle_num=sol_long.summary_variables["Cycle number"]
t = sol_long["Time [s]"].entries
I = sol_long["Current [A]"].entries
Q = sol_long['Discharge capacity [A.h]'].entries
LoC_SEI = sol_long["Loss of capacity to SEI [A.h]"].entries
N_SoC = sol_long["Negative electrode SOC"].entries
P_SoC = sol_long["Positive electrode SOC"].entries

ECM_whole_res=sol_long["Local ECM resistance [Ohm]"].entries
ECM_sum_res=sol_long.summary_variables["Local ECM resistance [Ohm]"]
ECM_sum_res_change=sol_long.summary_variables['Change in local ECM resistance [Ohm]']
cycle_time=sol_long.summary_variables['Time [s]']
# exp = 30e6*sol_0["Cell thickness change [m]"].entries
# exp_n = 30e6*sol_0["Negative Electrode thickness change [m]"].entries
# exp_p = 30e6*sol_0["Positive Electrode thickness change [m]"].entries
# ee_p = sol_0["X-averaged positive electrode porosity"].entries
# ee_n = sol_0["X-averaged negative electrode porosity"].entries
# ee_s = sol_0["X-averaged separator porosity"].entries
# j_n = sol_0["X-averaged negative electrode total interfacial current density [A.m-2]"].entries
# j_p = sol_0["X-averaged positive electrode total interfacial current density [A.m-2]"].entries

# x = sol_0["x [m]"].entries
# c_e_0 = sol_0["Electrolyte concentration [mol.m-3]"].entries

In [31]:
mdic = {"x100"+ident: x100, "y100"+ident:y100, "x0"+ident:x0,"y0"+ident:y0,"Cap"+ident:Cap,
        "LoC_SEI_summ"+ident:LoC_SEI_summ,
        "LAM_P"+ident:LAM_P,"LAM_N"+ident:LAM_N,"C_Neg"+ident:C_Neg,"C_pos"+ident:C_pos,
        "LLI"+ident:LLI,"cycle_num"+ident:cycle_num,
        "t"+ident:t,"LoC_SEI"+ident:LoC_SEI,"N_SoC"+ident:N_SoC,"P_SoC"+ident:P_SoC,
        "Q"+ident:Q, "I"+ident:I,    
        "ECM_whole_res":ECM_whole_res,"ECM_sum_res":ECM_sum_res,
        "ECM_sum_res_change":ECM_sum_res_change,"cycle_time":cycle_time 
       }
savemat("Outputs_Oct23_GM_DRC_DRC",mdic)

In [ ]:
# savemat("x100_test", mdic)


In [ ]:
# print(x100)

In [ ]:
# import csv
# with open(r"F:\Git Hub rep\CRC\neew\PyBaMM\Untitled Folder\whatever.csv", 'w') as f:
#     # create the csv writer
#     writer = csv.writer(f)

# write a row to the csv file
# i=1
# for lines in sol_long.all_summary_variables:
# #     writer.writerow(words)
#     i=i+1
#     savemat(f'{i:02d}', lines)


In [ ]:
type(x100)

In [ ]:
sol_long.variables.search("charge")

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Current [A]"].entries[1:15133])

In [ ]:
# plt.plot(sol_long["Time [s]"].entries[1:15133], sol_long["Terminal voltage [V]"].entries[1:15133])

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [ ]:
sol_long["Time [s]"].entries

In [ ]:
spm.variables.search("resistance")

In [26]:
sim_long.plot(["Local ECM resistance",
"Local ECM resistance [Ohm]",
               "Resistance [Ohm]"])


interactive(children=(FloatSlider(value=0.0, description='t', max=3133.5561681515674, step=31.335561681515674)…

In [ ]:
 plt.plot(sol_long["Time [s]"].entries, 1-sol_long["Discharge capacity [A.h]"].entries/5)

In [27]:
sol_long.summary_variables 

{'Minimum measured discharge capacity [A.h]': array([0.        , 0.00047819, 0.00095634, 0.0014349 , 0.0019138 ,
        0.00239309, 0.00287275, 0.0033529 , 0.00383349, 0.0043144 ,
        0.00479574, 0.00527743, 0.00575961, 0.00624212, 0.00672503,
        0.0072083 , 0.00769208, 0.00817621, 0.00866076, 0.00914581,
        0.00963119, 0.01011698, 0.01060309, 0.01108973, 0.01157675,
        0.01206418, 0.01255202, 0.01304026, 0.01352891, 0.01401802,
        0.01450753, 0.01499742, 0.01548771, 0.01597844, 0.01646959,
        0.01696123, 0.01745326, 0.01794571, 0.0184385 , 0.01893177,
        0.01942544, 0.01991949, 0.02041411, 0.02090904, 0.02140442,
        0.02190019, 0.02239642, 0.02289298, 0.02339001, 0.02388748,
        0.02438538, 0.02488367, 0.02538249, 0.02588175, 0.02638145,
        0.02688157, 0.02738211, 0.02788303, 0.02838448, 0.02888636,
        0.02938857, 0.02989137, 0.03039455, 0.03089816, 0.03140227,
        0.03190675, 0.03241165, 0.03291703, 0.03342292, 0.03392914,
   